# Import packages

In [1]:
from keras.backend.tensorflow_backend import set_session
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, AveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from keras import optimizers
from keras import backend as K
from keras.utils import to_categorical
import tensorflow as tf

import glob
import imageio
import pandas as pd
import os
import re
import numpy as np

/home/ubunt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load data & preprocessing

In [4]:
# path 설정
folder_path = '/home/ubunt/wav2mid/data/cnn inputs4/*'
remove_len = len('/home/ubunt/wav2mid/data/cnn inputs4/')
remove_len

37

In [5]:
# label 개수
a=glob.glob(folder_path + "*")
len(a)

74

In [9]:
# check
print(a[0],'\n',a[1])

/home/ubunt/wav2mid/data/cnn inputs4/52_71 
 /home/ubunt/wav2mid/data/cnn inputs4/45


In [10]:
# label list 생성
class_list = []  
for i in range(len(a)) :
    class_list.append(a[i][remove_len:])
print(class_list)

['52_71', '45', '68_71', '85', '59_68', '61_76', '54_73_81', '59_71', '54_69', '47_81', '52_73', '64_81', '42_69', '62', '49_76', '62_81', '74', '73', '59_74', '57', '80', '78', '64', '86', '52_74', '61_88', '57_76', '68', '69', '54', '52_64_68_71', '61', '59', '45_49', '42_45', '54_61_69', '52', '62_73', '45_74', '45_76', '83', '40_80', '42_81', '52_83', '64_85', '52_81', '52_68', '57_61_69', '66', '61_81', '57_69_76', '45_81', '50_73', '47_71', '47_76', '61_80', '45_64_73', '54_81', '57_69', '57_81', '66_81', '49', '81', '59_81', '49_81', '57_74', '52_80', '52_64_66_71', '50', '71', '45_73', '50_81', '59_76', '52_85']


In [11]:
# 총 train data 개수
val_len_list = 0
for i in range(len(a)) :
    val_len_list += int(len(glob.glob(a[i] + "/*.jpg")))

In [12]:
val_len_list

13026

In [15]:
# 모든 이미지가 저장
dataset=[] 
# 88개 one-hot label
Y_list=[]
# 88개 음에 대한 리스트.21~108
list88 = [str(i) for i in range(21,109)] 
# 각 음 폴더에 사진이 몇 장들어가는지 (몇 번 등장하는지)
class_len_list=[] 

for i in range(len(class_list)):
    Y=np.zeros(len(list88))
    #i번 폴더의 이름에 들어가는 숫자를 전부 추출 (22_42 -> 22,42)
    find=re.findall("[0-9]+", class_list[i]) 
    
    # 해당 숫자 one-hot encoding
    for k in range(len(find)):
        Y[list88.index(find[k])] = 1

    # 각 라벨 폴더에서 이미지 가져오가 
    for image_path in glob.glob(a[i]+"/*.jpg"): 
        # image load
        im = imageio.imread(image_path)
        # append
        Y_list.append(Y)
        dataset.append(im)
    
    # 해당 폴더의 이미지 개수 for KNN
    class_len_list.append(len(glob.glob(a[i]+"/*.jpg")))
    print('dataset'+ str(a[i][remove_len:])+'complete!')
print('file_num :',len(dataset))
dataset = np.array(dataset)

dataset52_71complete!
dataset45complete!
dataset68_71complete!
dataset85complete!
dataset59_68complete!
dataset61_76complete!
dataset54_73_81complete!
dataset59_71complete!
dataset54_69complete!
dataset47_81complete!
dataset52_73complete!
dataset64_81complete!
dataset42_69complete!
dataset62complete!
dataset49_76complete!
dataset62_81complete!
dataset74complete!
dataset73complete!
dataset59_74complete!
dataset57complete!
dataset80complete!
dataset78complete!
dataset64complete!
dataset86complete!
dataset52_74complete!
dataset61_88complete!
dataset57_76complete!
dataset68complete!
dataset69complete!
dataset54complete!
dataset52_64_68_71complete!
dataset61complete!
dataset59complete!
dataset45_49complete!
dataset42_45complete!
dataset54_61_69complete!
dataset52complete!
dataset62_73complete!
dataset45_74complete!
dataset45_76complete!
dataset83complete!
dataset40_80complete!
dataset42_81complete!
dataset52_83complete!
dataset64_85complete!
dataset52_81complete!
dataset52_68complete!
datas

In [16]:
# Final label *
y = np.array(Y_list)
Y = np.eye(3)[y.astype(int)]

change_y=np.array(Y_list)
change_Y=np.eye(3)[change_y.astype(int)]

In [17]:
# 확인
dataset.shape

(13026, 224, 224, 3)

# Modeling

In [18]:
# Optimizer
Adam = optimizers.Adam()

In [23]:
# Modeling * 전이학습 비교 필요
classifier = Sequential()

classifier.add(Conv2D(16, (3, 3), activation = 'relu',kernel_initializer='he_normal',input_shape = (224,224,3))) # 222 * 222
classifier.add(BatchNormalization())
classifier.add(Conv2D(16, (3, 3), activation = 'relu',kernel_initializer='he_normal')) # 220 * 220
classifier.add(BatchNormalization())
classifier.add(Conv2D(16, (3, 3), activation = 'relu',kernel_initializer='he_normal')) # 218 * 218
classifier.add(BatchNormalization())
classifier.add(Conv2D(16, (3, 3), activation = 'relu',kernel_initializer='he_normal')) # 216 * 216
classifier.add(BatchNormalization())

classifier.add(Conv2D(16, (3, 3), activation = 'relu',kernel_initializer='he_normal')) # 214 * 214
classifier.add(BatchNormalization())
classifier.add(Conv2D(88, (3, 3), activation = 'relu',kernel_initializer='he_normal')) # 212 * 212
classifier.add(BatchNormalization())
classifier.add(AveragePooling2D(pool_size=(106,106)))

classifier.add(Flatten())
classifier.add(Dense(units = 88, activation = 'sigmoid'))
classifier.compile(optimizer = Adam, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [24]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 222, 222, 16)      448       
_________________________________________________________________
batch_normalization_7 (Batch (None, 222, 222, 16)      64        
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 220, 220, 16)      2320      
_________________________________________________________________
batch_normalization_8 (Batch (None, 220, 220, 16)      64        
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 218, 218, 16)      2320      
_________________________________________________________________
batch_normalization_9 (Batch (None, 218, 218, 16)      64        
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 216, 216, 16)      2320      
__________

In [15]:
# 가중치 가져오기
# classifier.load_weights("/home/ubunt/wav2mid/data/weights/w_88layer_pickle1_10_pickle2_10_pickle3_10_pickle4_10_pickle5_3.h5")

In [25]:
# fitting
classifier.fit(dataset,y, epochs=15, batch_size = 32,shuffle=True)

Epoch 1/15
  544/13026 [>.............................] - ETA: 3:06 - loss: 0.6641 - acc: 0.6810

KeyboardInterrupt: 

In [22]:
# 가중치 저장
classifier.save_weights("weight_58231_37087_1106_01.hd5")

In [17]:
# Predict
pred = classifier.predict_proba(dataset)
pred = np.array(pred)
# 
pred.shape

(13026, 88)

# Best threshold 설정

Multi label classification 문제여서 마지막 layer에 softmax 대신 sigmoid를 적용하기 때문에

각각의 음에 대한 최적의 임계치를 설정해야한다.

In [26]:
from sklearn.metrics import matthews_corrcoef

In [20]:
# threshold 탐색 범위 설정
threshold = np.arange(0.01,np.max(pred),0.025)
# 변수 선언
acc = []
accuracies = []
best_threshold = np.zeros(pred.shape[1])
# 탐색
for i in range(pred.shape[1]):
    print(i)
    # 모델의 예측값
    y_prob = np.array(pred[:,i])
    # 설정된 범위의 threshold 적용
    for j in threshold:
        y_pred = [1 if prob>=j else 0 for prob in y_prob]
        # 해당 threshold의 정확도
        acc.append(matthews_corrcoef(y[:,i],y_pred))
    acc = np.array(acc)
    # max acc
    index = np.where(acc==acc.max())
    accuracies.append(acc.max())
    best_threshold[i] = threshold[index[0][0]]
    acc = []
print("best thresholds", best_threshold)

0


/home/ubunt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:538: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
best thresholds [0.01  0.01  0.01  0.01  0.01  0.01  0.01  0.01  0.01  0.01  0.01  0.01
 0.01  0.01  0.01  0.01  0.01  0.01  0.01  0.06  0.01  0.035 0.01  0.01
 0.035 0.01  0.035 0.01  0.035 0.035 0.01  0.035 0.01  0.035 0.01  0.01
 0.035 0.01  0.035 0.01  0.035 0.035 0.01  0.11  0.01  0.01  0.01  0.035
 0.06  0.01  0.01  0.01  0.11  0.035 0.01  0.01  0.01  0.035 0.01  0.01
 0.06  0.01  0.035 0.01  0.035 0.01  0.01  0.01  0.01  0.01  0.01  0.01
 0.01  0.01  0.01  0.01  0.01  0.01  0.01  0.01  0.01  0.01  0.01  0.01
 0.01  0.01  0.01  0.01 ]


In [22]:
# path
threshold_path = "/home/ubunt/wav2mid/threshold/"

In [24]:
# 예측값 csv 저장
pd.DataFrame(pred).to_csv(threshold_path + "06_acc1_88layer_pred.csv")

In [25]:
# 해당 예측값 best threshold csv 저장
pd.DataFrame(best_threshold).to_csv(threshold_path + "06_acc1_88layer_threshold.csv")

In [27]:
# best threshold 적용된 최종 예측값
y_pred = np.array([[1 if pred[i,j]>=best_threshold[j] else 0 for j in range(y[:,:].shape[1])] for i in range(y[:,:].shape[0])])

In [28]:
y_pred[0,:]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [46]:
# 정답과 예측 비교
i = 10
print(np.where(y_pred[i,:] == 1))
print(np.where(y[i,:] == 1))

(array([43, 46, 61]),)
(array([43, 46, 61]),)
